In [ ]:
import gensim
import os

In [ ]:
import pandas as pd
devDs = pd.read_csv('//content/CT24_checkworthy_dutch_dev-test.tsv', sep = '\t')
testtDs = pd.read_csv('/content/CT24_checkworthy_dutch_dev.tsv', sep = '\t')
trainDs = pd.read_csv('/content/CT24_checkworthy_dutch_train.tsv', sep = '\t')
testDs = pd.read_csv('/content/CT24_checkworthy_dutch_test_gold.tsv', sep = '\t')

df = pd.concat([devDs, testtDs, trainDs], ignore_index = True)
df.head()

,tweet_id,tweet_url,tweet_text,class_label
0,1256120472875524096,http://twitter.com/user/status/125612047287552...,"Ik merk op da ""kleinere"" supermark en me voo...",Yes
1,1256154134027350019,http://twitter.com/user/status/125615413402735...,Vanaf maandag 4 mei dragen we een mondmasker o...,No
2,1256253220239872000,http://twitter.com/user/status/125625322023987...,@evaverhoevenph1 Er is bij mijn we en geen enk...,Yes
3,1256305038139576326,http://twitter.com/user/status/125630503813957...,De #Volkskran 29 april: In #Nederland zijn b...,No
4,1256487670311682048,http://twitter.com/user/status/125648767031168...,Hoe ro en ongewens de si ua ie waarin we me ...,No


In [ ]:
no_indices = df[df['class_label'] == "NO"].index[:100]  # Select only the first 100 indices
df = df.drop(no_indices)
df['class_label'].value_counts()

class_label
No     1090
Yes     823
Name: count, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

story = []
for index, row in df.iterrows():
  text = row['tweet_text']
  raw_sent = sent_tokenize(text)
  for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
word2vecModel = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
word2vecModel.build_vocab(story)

In [ ]:
word2vecModel.train(story, total_examples = word2vecModel.corpus_count, epochs = word2vecModel.epochs)

(174760, 276755)

In [ ]:
df['class_label'] = df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)
df2 = testDs
df2['class_label'] = df2['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, GRU, MultiHeadAttention, LayerNormalization, Concatenate, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import RandomOverSampler

def get_word_embedding_model(word_embedding_type):
    if word_embedding_type == "word2vec":
        word_embedding_model = word2vecModel
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")
    return word_embedding_model

def get_sentence_embedding(sentence, model, word_embedding_type):
    if word_embedding_type == "word2vec":
        tokens = word_tokenize(sentence.lower())
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if not vectors:
            return np.zeros(model.vector_size)
        return np.mean(vectors, axis=0)
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")

def create_embedding_matrix(word_index, word_embedding_model, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word_embedding_model.wv:
            embedding_matrix[i] = word_embedding_model.wv[word]
        else:
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix

def create_stacked_lstm_model(embedding_matrix, input_length, model_type):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=False))
    model.add(SpatialDropout1D(0.2))

    if model_type == "lstm":
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "bilstm":
        model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "gru":
        model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(GRU(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    else:
        raise ValueError("Unsupported model type. Choose 'lstm', 'bilstm', or 'gru'.")

    query = model.output
    key = model.output
    value = model.output

    attention_layer = MultiHeadAttention(num_heads=4, key_dim=50)
    attention_output = attention_layer(query=query, value=value, key=key)
    attention_output = LayerNormalization()(attention_output)

    # Concatenate LSTM output and attention output
    combined_output = Concatenate()([model.output, attention_output])

    # Apply GlobalAveragePooling1D to aggregate the sequence into a single vector
    combined_output = GlobalAveragePooling1D()(combined_output)

    combined_output = Dense(1, activation='sigmoid')(combined_output)

    final_model = Model(inputs=model.input, outputs=combined_output)
    final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return final_model

def train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type):
    y_train = train_data["class_label"].values
    y_dev = dev_data["class_label"].values

    word_embedding_model = get_word_embedding_model(word_embedding_type)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_data["tweet_text"].tolist() + dev_data["tweet_text"].tolist())
    X_train_sequences = tokenizer.texts_to_sequences(train_data["tweet_text"].tolist())
    X_dev_sequences = tokenizer.texts_to_sequences(dev_data["tweet_text"].tolist())

    word_index = tokenizer.word_index
    embedding_matrix = create_embedding_matrix(word_index, word_embedding_model, word_embedding_model.vector_size)

    max_sequence_length = max(len(seq) for seq in X_train_sequences + X_dev_sequences)
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
    X_dev_padded = pad_sequences(X_dev_sequences, maxlen=max_sequence_length)

    # Handle data imbalance using random oversampling
    ros = RandomOverSampler(random_state=42)
    X_train_padded_resampled, y_train_resampled = ros.fit_resample(X_train_padded, y_train)

    stacked_lstm_model = create_stacked_lstm_model(embedding_matrix, max_sequence_length, model_type)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    stacked_lstm_model.fit(X_train_padded_resampled, y_train_resampled, epochs=5, batch_size=32,
                           validation_data=(X_dev_padded, y_dev),
                           callbacks=[early_stopping])

    dev_predictions = (stacked_lstm_model.predict(X_dev_padded, verbose=0) > 0.5).astype("int32").flatten()
    y_dev = y_dev.flatten()
    f1score = f1_score(y_dev, dev_predictions)

    print(f"F1-score on Development Set: {f1score}")
    return stacked_lstm_model


In [ ]:
word_embedding_type = "word2vec"
model_type = "lstm"
train_data = df
dev_data = df2

stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
69/69 [==============================] - 95s 757ms/step - loss: 0.7062 - accuracy: 0.5463 - val_loss: 0.6669 - val_accuracy: 0.6030
Epoch 2/5
69/69 [==============================] - 42s 613ms/step - loss: 0.6902 - accuracy: 0.5344 - val_loss: 0.7172 - val_accuracy: 0.5330
Epoch 3/5
69/69 [==============================] - 42s 611ms/step - loss: 0.6926 - accuracy: 0.5477 - val_loss: 0.6826 - val_accuracy: 0.5690
Epoch 4/5
69/69 [==============================] - 41s 595ms/step - loss: 0.6841 - accuracy: 0.5546 - val_loss: 0.6436 - val_accuracy: 0.5870
Epoch 5/5
69/69 [==============================] - 40s 581ms/step - loss: 0.6869 - accuracy: 0.5505 - val_loss: 0.6828 - val_accuracy: 0.5680
F1-score on Development Set: 0.5992578849721707


NameError: name 'stacked_lstm_mode' is not defined

In [ ]:
word_embedding_type = "word2vec"
model_type = "bilstm"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
69/69 [==============================] - 82s 937ms/step - loss: 0.7045 - accuracy: 0.5436 - val_loss: 0.7990 - val_accuracy: 0.3980
Epoch 2/5
69/69 [==============================] - 58s 839ms/step - loss: 0.7042 - accuracy: 0.5243 - val_loss: 0.6516 - val_accuracy: 0.6060
Epoch 3/5
69/69 [==============================] - 61s 890ms/step - loss: 0.6849 - accuracy: 0.5596 - val_loss: 0.7230 - val_accuracy: 0.5280
Epoch 4/5
69/69 [==============================] - 61s 878ms/step - loss: 0.6863 - accuracy: 0.5550 - val_loss: 0.7346 - val_accuracy: 0.4740
Epoch 5/5
69/69 [==============================] - 65s 929ms/step - loss: 0.6941 - accuracy: 0.5257 - val_loss: 0.6704 - val_accuracy: 0.5900
F1-score on Development Set: 0.42397660818713445


In [ ]:
word_embedding_type = "word2vec"
model_type = "gru"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
69/69 [==============================] - 39s 432ms/step - loss: 0.7053 - accuracy: 0.5491 - val_loss: 0.6953 - val_accuracy: 0.5130
Epoch 2/5
69/69 [==============================] - 29s 424ms/step - loss: 0.6862 - accuracy: 0.5440 - val_loss: 0.6635 - val_accuracy: 0.5650
Epoch 3/5
69/69 [==============================] - 29s 422ms/step - loss: 0.6851 - accuracy: 0.5573 - val_loss: 0.6885 - val_accuracy: 0.5260
Epoch 4/5
69/69 [==============================] - 29s 424ms/step - loss: 0.6830 - accuracy: 0.5725 - val_loss: 0.7189 - val_accuracy: 0.4830
Epoch 5/5
69/69 [==============================] - 30s 443ms/step - loss: 0.6867 - accuracy: 0.5601 - val_loss: 0.6631 - val_accuracy: 0.5890
F1-score on Development Set: 0.5714285714285714
